In [112]:
import numpy as np
import cv2
import torch
from image_ops import load_and_resize, preprocess_im, pil_bgr_to_rgb, combine_image_and_heatmap, combine_horz
from similarity_ops import compute_spatial_similarity
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchvision.models import resnet18

In [138]:
type1 = 'faces'
type2 = 'hotels'

In [139]:
model = resnet18(pretrained=True).eval()

# Get your input
img1 = read_image(type1 + "1.jpg")
img2 = read_image(type2 + "2.jpg")

# Preprocess
img1_norm = normalize(resize(img1, (224, 224)) / 255., [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
img2_norm = normalize(resize(img2, (224, 224)) / 255., [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

/home/shreyas/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/shreyas/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/shreyas/anaconda3/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, 

In [140]:
f = torch.nn.Sequential(*list(model.children())[:-2])  
features1 = f(img1_norm.unsqueeze(0))
features2 = f(img2_norm.unsqueeze(0))

In [141]:
c, h, w = features1.squeeze(0).shape

In [142]:
# Compute the similarity heatmap
conv1 = features1.squeeze(0).permute(1, 2, 0).detach().numpy().reshape(h*w, c)
conv2 = features2.squeeze(0).permute(1, 2, 0).detach().numpy().reshape(h*w, c)
similarity = compute_spatial_similarity(conv1, conv2)

In [143]:
similarity1, similarity2 = similarity

In [144]:
similarity1

array([[0.00230161, 0.00375747, 0.00388819, 0.0057789 , 0.00650663,
        0.00912857, 0.00840714],
       [0.0036832 , 0.00643441, 0.00661743, 0.00961917, 0.01025735,
        0.01443241, 0.01251629],
       [0.00540263, 0.00750655, 0.00594269, 0.00773212, 0.00922846,
        0.01459241, 0.01406103],
       [0.00732416, 0.00944719, 0.00692369, 0.00806311, 0.00925338,
        0.01417573, 0.01366571],
       [0.00891512, 0.01091807, 0.00905613, 0.00905808, 0.0084312 ,
        0.0113091 , 0.01077592],
       [0.0099806 , 0.01516167, 0.02008093, 0.02404412, 0.02107863,
        0.01740684, 0.0108393 ],
       [0.00839548, 0.01214599, 0.01583638, 0.01941086, 0.01781347,
        0.0133991 , 0.00804534]])

In [145]:
img1_path = type1 + "1.jpg"
img2_path = type2 + "2.jpg"

# Load the images
img1_arr = load_and_resize(img1_path)
img2_arr = load_and_resize(img2_path)

In [146]:
img1_out = combine_image_and_heatmap(img1_arr, similarity1)
img2_out = combine_image_and_heatmap(img2_arr, similarity2)

In [147]:
sim_final = combine_horz([img1_out, img2_out])

In [148]:
sim_final_pil = Image.fromarray(np.uint8(sim_final))
sim_bgr2rgb = pil_bgr_to_rgb(sim_final_pil)

In [149]:
sim_path = "sim_{}_{}.jpg".format(type1, type2)
sim_bgr2rgb.save(sim_path)